In [1]:
import numpy as np
import scipy.misc
from scipy.fftpack import dct, idct
import sys
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import random
from tqdm._tqdm_notebook import tqdm_notebook
from scipy.fftpack import dct, idct
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import sympy

%matplotlib inline


class ImageLoader:
    def __init__(self, FILE_PATH):
        self.img = np.array(Image.open(FILE_PATH))
        # 行数
        self.row_blocks_count = self.img.shape[0] // 8
        # 列数
        self.col_blocks_count = self.img.shape[1] // 8

    def get_points(self, POINT):
        Row = random.randint(0, len(self.img) - POINT - 1)
        Col = random.randint(0, len(self.img) - 1)
        return self.img[Row : Row + POINT, Col]

    def get_block(self, col, row):
        return self.img[col * 8 : (col + 1) * 8, row * 8 : (row + 1) * 8]


# plt.rcParams['font.family'] ='sans-serif'#使用するフォント
# plt.rcParams["font.sans-serif"] = "Source Han Sans"
plt.rcParams["font.family"] = "Source Han Sans JP"  # 使用するフォント
plt.rcParams["xtick.direction"] = "in"  # x軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams["ytick.direction"] = "in"  # y軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams["xtick.major.width"] = 1.0  # x軸主目盛り線の線幅
plt.rcParams["ytick.major.width"] = 1.0  # y軸主目盛り線の線幅
plt.rcParams["font.size"] = 12  # フォントの大きさ
plt.rcParams["axes.linewidth"] = 1.0  # 軸の線幅edge linewidth。囲みの太さ
matplotlib.font_manager._rebuild()

MONO_DIR_PATH = "../../Mono/"
AIRPLANE = ImageLoader(MONO_DIR_PATH + "airplane512.bmp")
BARBARA = ImageLoader(MONO_DIR_PATH + "barbara512.bmp")
BOAT = ImageLoader(MONO_DIR_PATH + "boat512.bmp")
GOLDHILL = ImageLoader(MONO_DIR_PATH + "goldhill512.bmp")
LENNA = ImageLoader(MONO_DIR_PATH + "lenna512.bmp")
MANDRILL = ImageLoader(MONO_DIR_PATH + "mandrill512.bmp")
MILKDROP = ImageLoader(MONO_DIR_PATH + "milkdrop512.bmp")
SAILBOAT = ImageLoader(MONO_DIR_PATH + "sailboat512.bmp")

IMAGES = [
    AIRPLANE,
    BARBARA,
    BOAT,
    GOLDHILL,
    LENNA,
    MANDRILL,
    MILKDROP,
    SAILBOAT
]

In [2]:
# n_bar = 4
N = 32

In [3]:
IMG = LENNA

In [4]:
Fk = np.zeros(IMG.img.shape)

In [5]:
sampling_x = (0.5 + np.arange(N)) / N
u_1 = (1 - sampling_x) ** 2 / 2
u_2 = 1 / 24 - sampling_x ** 2 / 6 + sampling_x ** 3 / 6 - sampling_x ** 4 / 24
u_3 = sampling_x ** 2 / 2
u_4 = -sampling_x ** 2 / 12 + sampling_x ** 4 / 24

In [6]:
ak = - scipy.fftpack.dct(u_1,norm="ortho")
bk = scipy.fftpack.dct(u_2,norm="ortho")
ck = scipy.fftpack.dct(u_3,norm="ortho")
dk = scipy.fftpack.dct(u_4,norm="ortho")

alpha = ak
beta = ck

Ak = (2 * ak - 16 * bk) / np.sqrt(N)
Bk = (2 * ck - 16 * dk) / np.sqrt(N)
Ck = (2 * ak - 32 * bk) / np.sqrt(N)
Dk = (2 * ck - 32 * dk) / np.sqrt(N)

In [7]:
def TPHLCT(IMG,N):
    Fk = np.zeros(IMG.img.shape)
    # DCT
    for row in range(IMG.img.shape[0] // N):
        for col in range(IMG.img.shape[1]):
            eight_points = IMG.img[N * row : N * (row + 1), col]
            c = scipy.fftpack.dct(eight_points,norm="ortho")
            Fk[N * row : N * (row + 1), col] = c
            
    # 3階
    for row in range(Fk.shape[0] // N):
        for col in range(Fk.shape[1]):
            F = Fk[N * row : N * (row + 1), col]

            F_0_r = 0
            F_1_r = 0
            if row is not Fk.shape[0] // N - 1:
                F_0_r = Fk[N * (row + 1), col]
                F_1_r = Fk[N * (row + 1) + 1, col]

            F_0_l = 0
            F_1_l = 1
            if row is not 0:
                F_0_l = Fk[N * (row - 1), col]
                F_1_l = Fk[N * (row - 1) + 1, col]

            # 残差
            F_0 = F[0]
            F_1 = F[1]

            F = (
                F
                - Ak * (F_0 - F_0_l)
                - Bk * (F_0_r - F_0)
                - Ck * (F_1 + F_1_l)
                - Dk * (F_1_r + F_1)
            )
            # F_0, F_1は残す
            F[0] = F_0
            F[1] = F_1
            # F_0 V_1 V_2 V_3 V_4 V_5 V_6 V_7
            Fk[N * row : N * (row + 1), col] = F
    
    # 1階
    for row in range(Fk.shape[0] // N):
        for col in range(Fk.shape[1]):
            F = Fk[N * row : N * (row + 1), col]

            F_0_r = 0
            if row is not Fk.shape[0] // N - 1:
                F_0_r = Fk[N * (row + 1), col]

            F_0_l = 0
            if row is not 0:
                F_0_l = Fk[N * (row - 1), col]

            # 残差
            F_0 = F[0]
            F_temp = F - alpha * (F_0_r - F_0) / np.sqrt(N) - beta * (F_0 - F_0_l) / np.sqrt(N)
            # F_0は残す
            F[1] = F_temp[1]
            # F_0 V_1 V_2 V_3 V_4 V_5 V_6 V_7
            Fk[N * row : N * (row + 1), col] = F
            
    # DCT
    for row in range(Fk.shape[0]):
        for col in range(Fk.shape[1] // N):
            eight_points = Fk[row, N * col : N * (col + 1)]
            c = scipy.fftpack.dct(eight_points,norm="ortho")
            Fk[row, N * col : N * (col + 1)] = c
            
    # 3階
    for row in range(Fk.shape[0]):
        for col in range(Fk.shape[1] // N):
            F = Fk[row, N * col : N * (col + 1)]

            F_0_r = 0
            F_1_r = 0
            if col is not Fk.shape[1] // N - 1:
                F_0_r = Fk[row, N * (col + 1)]
                F_1_r = Fk[row, N * (col + 1) + 1]

            F_0_l = 0
            F_1_l = 0
            if col is not 0:
                F_0_l = Fk[row, N * (col - 1)]
                F_1_l = Fk[row, N * (col - 1) + 1]

            # 残差
            F_0 = F[0]
            F_1 = F[1]

            F = (
                F
                - Ak * (F_0 - F_0_l)
                - Bk * (F_0_r - F_0)
                - Ck * (F_1 + F_1_l)
                - Dk * (F_1_r + F_1)
            )
            # F_0は残す
            F[0] = F_0
            F[1] = F_1
            # F_0 V_1 V_2 V_3 V_4 V_5 V_6 V_7
            Fk[row, N * col : N * (col + 1)] = F
            
    for row in range(Fk.shape[0]):
        for col in range(Fk.shape[1] // N):
            F = Fk[row, N * col : N * (col + 1)]

            F_0_r = 0
            if col is not Fk.shape[1] // N - 1:
                F_0_r = Fk[row, N * (col + 1)]

            F_0_l = 0
            if col is not 0:
                F_0_l = Fk[row, N * (col - 1)]

            # 残差
            F_0 = F[0]
            F_temp = F - alpha * (F_0_r - F_0) / np.sqrt(N) - beta * (F_0 - F_0_l) / np.sqrt(N)
            # F_0は残す
            F[1] = F_temp[1]
            # F_0 V_1 V_2 V_3 V_4 V_5 V_6 V_7
            Fk[row, N * col : N * (col + 1)] = F
            
    return Fk

In [8]:
# DCT係数の平均を求める

In [9]:
Fk_values = np.zeros((512,512))

In [10]:
for IMG in tqdm_notebook(IMAGES):
    values = np.zeros((25,4))
    Fk = np.zeros(IMG.img.shape)
    for row in range(IMG.img.shape[0] // N):
        for col in range(IMG.img.shape[1] // N):
            block = IMG.img[row * N : (row + 1) * N, col * N : (col + 1) * N]
            c = scipy.fftpack.dct(
                scipy.fftpack.dct(block, axis=0, norm="ortho"), axis=1, norm="ortho"
            )
            Fk[row * N : (row + 1) * N, col * N : (col + 1) * N] = c

    Fk_values += np.abs(Fk)

In [11]:
Fk_values /= len(IMAGES)
pd.DataFrame(Fk_values).to_csv("DCT_coef_ave.csv",header=False,index=False)

In [12]:
# 各n_barの残差係数の平均を求める

In [13]:
Vk_values = np.zeros((512,512))

In [14]:
Vk_values = np.zeros((512,512))
for IMG in tqdm_notebook(IMAGES):
    Fk = TPHLCT(IMG,N)
    Vk_values += np.abs(Fk)

pd.DataFrame(Vk_values / len(IMAGES)).to_csv("TPHLCT_coef_ave.csv",header=False,index=False)

In [15]:
# DCT係数を読み込む

In [16]:
Fk_values = pd.read_csv("DCT_coef_ave.csv",header=None).values

In [17]:
# 残差係数を読み込む

In [18]:
Vk_values = pd.read_csv("TPHLCT_coef_ave.csv",header=None).values

In [19]:
# NxNブロック1個当たりの係数の平均を求める

In [20]:
Fk_block_ave_values = np.zeros((N,N))
for row in range(IMG.img.shape[0] // N):
    for col in range(IMG.img.shape[1] // N):
        if col == 0:
            continue
        if col == IMG.img.shape[1] // N -1:
            continue
        if row == 0:
            continue
        if row == IMG.img.shape[0] // N -1:
            continue
        block = Fk_values[row * N : (row + 1) * N, col * N : (col + 1) * N]
        Fk_block_ave_values += np.abs(block)

Fk_block_ave_values /= (IMG.img.shape[0]//N)**2

In [21]:
Vk_block_ave_values = np.zeros((N,N))
for row in range(IMG.img.shape[0] // N):
    for col in range(IMG.img.shape[1] // N):
        if col == 0:
            continue
        if col == IMG.img.shape[1] // N -1:
            continue
        if row == 0:
            continue
        if row == IMG.img.shape[0] // N -1:
            continue
        block = Vk_values[row * N : (row + 1) * N, col * N : (col + 1) * N]
        Vk_block_ave_values += np.abs(block)

Vk_block_ave_values /= (IMG.img.shape[0]//N)**2

In [22]:
df = pd.DataFrame(columns=["G(k_1,k_2)"])

In [23]:
k_max = 5
for index in range(1,k_max,1):
    for i in range(index):
        Gk1k2_1 = []
        Gk1k2_2 = []
        Vk = Vk_block_ave_values[i,index]
        Fk = Fk_block_ave_values[i,index]
        Gk1k2_1.append(100 * (1 - Vk/Fk))        

        Vk = Vk_block_ave_values[index,i]
        Fk = Fk_block_ave_values[index,i]
        Gk1k2_2.append(100 * (1 - Vk/Fk))
            
        df.loc["(" + str(i) + "," + str(index) + ")"] = Gk1k2_1
        df.loc["(" + str(index) + "," + str(i) + ")"] = Gk1k2_2
        
    Gk1k2 = []

    Vk = Vk_block_ave_values[index,index]
    Fk = Fk_block_ave_values[index,index]
    Gk1k2.append(100 * (1 - Vk/Fk))
    
    df.loc["(" + str(index) + "," + str(index) + ")"] = Gk1k2

Gk1k2 = []

Vk_sum = 0
Fk_sum = 0
for row in range(N):
    for col in range(N):
        if row > k_max-1 or col > k_max-1:
            Vk_sum += Vk_block_ave_values[row,col]
            Fk_sum += Fk_block_ave_values[row,col]
Gk1k2.append(100 * (1 - Vk_sum/Fk_sum))
df.loc["others"] = Gk1k2

In [24]:
df.to_csv("TPHLCT_high_freq_comp.csv")
df

,"G(k_1,k_2)"
"(0,1)",13.304470
"(1,0)",18.936476
"(1,1)",14.089218
"(0,2)",16.379916
"(2,0)",15.882375
"(1,2)",14.480531
"(2,1)",17.177013
"(2,2)",13.307026
"(0,3)",6.849967
"(3,0)",8.234717
